In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
np.random.seed(42)
import pprint
import s3fs

import sys
sys.path.append("../../")
from robust_pca.benchmark import comparator
from robust_pca.imputations import models

In [3]:
path = "s3://aifluence-data2/data/02_intermediate/chatelet_pdt/H.parq"
df = pd.read_parquet(path, engine="pyarrow")

**data preparation**

In [4]:
def nan_rule(df, columns, hours):
    """replace nan values by zero if "true" zero (e.g. no train during this period)
    Parameters
    ----------
    df : pd.DataFrame
        "spatiotemporal" dataframe
    columns : List[str]
        list of column names we have to replace nan by zero
    hours : List[str]
        list of hours for which we know there is no train -> true zeros
    """
    data = df.copy()
    data.loc[data.index.get_level_values("datetime").hour.isin(hours), columns] = data.loc[data.index.get_level_values("datetime").hour.isin(hours), columns].fillna(0)
    return data


data = df.reset_index()
#del data["datetime"]
#data = data.rename(columns={"datetime_theo_pdt":"datetime"})
data = data.sort_values(by=["station", "datetime", "position"])
data = data.set_index(["line","station", "datetime", "direction"])
data = nan_rule(data, ["load"], [0, 1, 2, 3])
data = data.reset_index()

In [7]:
data = data.iloc[:8]

In [8]:
print(data[(data.position==2) & (data.n_coaches==2)]["load"].isna().sum())
print(data[(data.position==1) & (data.n_coaches==2)]["load"].isna().sum())

0
0


In [9]:
data[(data.n_coaches==2) & (data.load.isnull())]["position"].unique()

array([], dtype=float64)

**Comparator**

In [10]:
models_to_test = [
  # models.ImputeByMean(["datetime.dt.dayofweek", "datetime.dt.round('10min')"]),
  # models.ImputeByMedian(["datetime.dt.dayofweek", "datetime.dt.round('10min')"]), 
  # models.RandomImpute(),
  # models.ImputeLOCF(),
  # models.ImputeNOCB(),
  # models.ImputeByInterpolation(**{"method": "linear"}),
  # models.ImputeBySpline(),
  # models.ImputeKNN(),
  models.ImputeRPCA(
    method="temporal", 
    aggregate_time="10min", 
    multivariate=False,
    **{"n_rows":24*12*7, "maxIter":10, "tau":1, "lam":0.8}
    ),
#   models.ImputeProphet(**{
#               "weekly_seasonality": True, 
#               "yearly_seasonality": True,
#               "interval_width": 0.95,
#               })
# models.ImputeRPCA(),
]

search_params = {
  "ImputeKNN": {"k": {"min":2, "max":3, "type":"Integer"}},
  "ImputeProphet": {
          "daily_seasonality": {"categories": [True, False], "type":"Categorical"},
        },
  # "ImputeRPCA": {
  #   "lam": {"min": 0.5, "max": 1, "type":"Real"},
  #   "tau": {"min": 1, "max": 1.5, "type":"Real"},
  # }
}

line = "H"
station = "VALMONDOIS"
direction = "Paris"
dataset = data[(data["line"]==line) & (data["station"]==station) & (data["direction"]==direction)]
cols_to_impute = ["load"]

# dataset = data[(data["line"]==line) & (data["station"]==station) & (data["direction"]==direction)][["datetime", "load"]]
# dataset = dataset.set_index("datetime")
# dataset = dataset.squeeze()
# #dataset = dataset.resample("5min").agg(pd.Series.sum, skipna=False)
# dataset = dataset.resample('5min').agg(lambda x: np.nan if np.isnan(x).all() else np.sum(x) )
# print(dataset.isna().sum())
# dataset = dataset.to_frame()

dataset = dataset.set_index(["line","station", "datetime", "direction"])
# dataset = dataset.set_index(["datetime", "direction"])
# dataset = dataframe with multiindex and at least, columns in cols_to_impute

prop_nan = 0.05
filter_value_nan = 20

In [11]:
comparison = comparator.Comparator(
          dataset,
          prop_nan, 
          models_to_test, 
          cols_to_impute,
          search_params=search_params,
          filter_value_nan=filter_value_nan,
)
results = comparison.compare()

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(results)

ImputeRPCA


KeyError: 'agg_values'